In [1]:
import pandas as pd
from lexas import evaluation
from lexas import prediction
symbols = prediction.symbols
mode = "all_following" #or just_next


# Preparation

In [51]:
path_to_csv = "./data/experiments_for_xgboost.csv"
train_dic = evaluation.generate_dic_for_eval(
    path_to_csv,
    1930,  #start year
    2018,  #end year
    mode) 

dev_dic = evaluation.generate_dic_for_eval(path_to_csv,2019,2019,mode)
test_dic = evaluation.generate_dic_for_eval(path_to_csv,2020,2023,mode)

#Save the dictionaries
#The disctionaries are saved as ./eval/dic_{mode}.csv
evaluation.save_dic(train_dic,dev_dic,test_dic,mode)

793it [00:00, 427682.02it/s]
793it [00:00, 1089804.41it/s]
793it [00:00, 606728.03it/s]


# Evaluation

In [2]:
#Loading data
train_dic,dev_dic,test_dic = evaluation.load_dic(mode)

print(train_dic["PLK4"]) #Genes examined after PLK4 in the articles published before 2018
print(dev_dic["PLK4"]) #Genes examined after PLK4 in the articles published in 2019
print(test_dic["PLK4"]) #Genes examined after PLK4 in the articles published after 2020

['PLK4', 'SASS6', 'STIL', 'NEK7']
['']
['PLK4', 'STIL', 'CEP152']


In [3]:
# Genes examined after a query gene in previous data are not considered false, 
# so they are removed from evaluation.
# prev_dic stores genes examined previously to the answer data set

eval_mode = "test" # dev or test

from sklearn.metrics import roc_auc_score
import numpy as np
if eval_mode=="dev":
        prev_dic = train_dic
        answer_dic = dev_dic
elif eval_mode=="test":
        prev_dic = {k:train_dic[k] + dev_dic[k] for k in train_dic}
        answer_dic = test_dic
else:
    raise Exception("Evaluation mode should be \"dev\" or \"test\"")

In [5]:
#Calculate AUC
model_name ="xgboost"
genes = ["PLK4","SASS6","CEP152","CEP192","CEP63","PCNT"]
result_dir = "./result/"
top_k = 100

import os
import tqdm

evaluation.calculate_auc_for_many_genes(result_dir, model_name, prev_dic, answer_dic, top_k, genes=genes)

100%|██████████| 6/6 [00:00<00:00, 13.37it/s]


,Symbol,AUC at 100 for xgboost
0,PLK4,0.748866
1,SASS6,NaN
2,CEP152,0.999738
3,CEP192,0.497798
4,CEP63,NaN
5,PCNT,0.999826
